In [2]:
!pip install streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 32.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=c5d5c81ff1982a88afbe30915da777a2f0efc625937d0665d2473b23e21f6f41
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: tzlocal
    Found existing installation: tzlocal 5.0.1
    Uninstalling tzloca

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, model_selection
import streamlit as st
import pandas as pd
import numpy as np
import joblib
labelencoder = preprocessing.LabelEncoder()
scaler = preprocessing.StandardScaler()
data = pd.read_csv("insurance.csv")
data.drop_duplicates(inplace=True)
data = data[["age","sex","bmi","children","smoker","charges"]]
data["sex"] = labelencoder.fit_transform(data["sex"])
data["smoker"] = labelencoder.fit_transform(data["smoker"])
X = data[["age","sex","bmi","children","smoker"]]
y= data["charges"]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,y, test_size = 0.3)
scaled_X_train = scaler.fit_transform(X_train)

In [4]:
X.head()

,age,sex,bmi,children,smoker
0,19,0,27.900,0,1
1,18,1,33.770,1,0
2,28,1,33.000,3,0
3,33,1,22.705,0,0
4,32,1,28.880,0,0


In [5]:
y.head()

0    16884.92400
1     1725.55230
2     4449.46200
3    21984.47061
4     3866.85520
Name: charges, dtype: float64

In [6]:
from sklearn.ensemble import RandomForestRegressor
rfr_model = RandomForestRegressor(max_features = 3, n_estimators = 128)
rfr_model.fit(scaled_X_train,y_train)


RandomForestRegressor(max_features=3, n_estimators=128)

In [7]:
import joblib
joblib.dump(rfr_model,"rfr_model")

['rfr_model']

In [8]:
%%writefile requirements.txt
pandas == 1.5.3
numpy == 1.22.4
joblib == 1.2.0
streamlit == 1.24.0

Writing requirements.txt


In [9]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib

# Load the trained model
model = joblib.load("/content/rfr_model")

# Define the function to preprocess user input
def preprocess_input(age, sex, bmi, children, smoker):
    # Perform data preprocessing
    data = pd.DataFrame({
        'age': [age],
        'sex': [sex],
        'bmi': [bmi],
        'children': [children],
        'smoker': [smoker]
    })
    return data

# Define the function to predict insurance charge
def predict_insurance_charge(data):
    # Use the loaded model to make predictions
    prediction = model.predict(data)
    return prediction

# Create the Streamlit web app
def main():
    # Set the title and sidebar
    st.title("Insurance Charge Estimation")
    st.sidebar.title("User Input")

    # Add sliders/inputs for user input
    age = st.sidebar.slider("Age", 20, 100, step=1, value=30)
    sex = st.sidebar.selectbox("Sex", [0, 1], format_func=lambda x: "Male" if x == 0 else "Female")
    bmi = st.sidebar.slider("BMI", 10.0, 40.0, step=0.1, value=20.0)
    children = st.sidebar.slider("Number of Children", 0, 10, step=1, value=0)
    smoker = st.sidebar.selectbox("Smoker", [0, 1], format_func=lambda x: "No" if x == 0 else "Yes")

    # Preprocess the input
    input_data = preprocess_input(age, sex, bmi, children, smoker)

    # Make predictions
    prediction = predict_insurance_charge(input_data)

    # Display the prediction
    st.subheader("Estimated Insurance Charge:")
    result_placeholder = st.empty()
    result_placeholder.write(prediction[0])


if __name__ == "__main__":
    # Launch the app in Streamlit
    main()


Writing app.py


In [10]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.138.172.162
npx: installed 22 in 4.674s
your url is: https://tasty-streets-follow.loca.lt
